In [ ]:
import os
from collections import defaultdict

import pandas as pd

from db.mongo import MyMongo

In [ ]:
with MyMongo() as db:
    url_script = db.find('public_website', 'a1_pages')

without_js_script = []

for item in url_script:
    item.pop('jsScript')
    without_js_script.append(item)

with MyMongo() as db:
    db.delete_and_insert('public_website', 'a2_js_files', without_js_script)

domain_unique_js_file = defaultdict(set)

for item in without_js_script:
    domain = item['netLoc']
    if item['jsFile']:
        js_files = set(item['jsFile'])
        domain_unique_js_file[domain] = domain_unique_js_file[domain].union(js_files)

df_data = []
for domain, unique_files in domain_unique_js_file.items():
    for file in unique_files:
        df_data.append((domain, file))

df_files_per_domain = pd.DataFrame(columns=['netLoc', 'jsFile', 'filePath'], data=df_data)
df_files_per_domain['path'] = os.path.join(df_files_per_domain['netLoc'], df_files_per_domain['jsFile'])

with MyMongo() as db:
    db.delete_and_insert_df('public_website', 'a3_js_files_per_domain', df_files_per_domain)